In [11]:
pip install cobra

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install --upgrade metDataModel

Requirement already up-to-date: metDataModel in /Users/pbais/opt/anaconda3/lib/python3.7/site-packages (0.3.1)
Note: you may need to restart the kernel to use updated packages.


# https://cobrapy.readthedocs.io/en/latest/io.html#SBML
import cobra

from metDataModel.core import Compound, Reaction, Pathway, metabolicModel

In [10]:
import os
import cobra
import json
import pickle

In [ ]:

from metDataModel.core import Compound, Reaction, Pathway, metabolicModel

def metabolite2compound(M):
    # convert cobra Metabolite to metDataModel Compound
    Cpd = Compound()
    Cpd.src_id = M.id
    Cpd.id = M.id.split("[")[0]
    Cpd.name = M.name
    Cpd.charge = M.charge
    Cpd.charged_formula = M.formula
    Cpd.db_ids = M.notes
    mydict = M.notes   ## other databaseIDs  are in the notes tag
    #print(M.notes['SMILES'])
    Cpd.SMILES= mydict.get("SMILES",None)
    Cpd.inchi= mydict.get("INCHI",None)
    
    return Cpd


## this is model dependent, as some models use symbols other than "[" !!!
def mclean(x): return x.split("[")[0]


# port reactions
def port_reaction(R):
    new = Reaction()
    new.id = R.id
    new.reactants = [mclean(m.id) for m in R.reactants] 
    new.products = [mclean(m.id) for m in R.products] 
    return new


# pathways, using group as pathway from EBI. Other models may use subsystem etc.
def port_pathway(P):
    new = Pathway()
    new.id = P.id
    new.source = ['EBI',]
    new.name = P.name
    new.list_of_reactions = [x.id for x in P.members]
    return new

# source data is cloned from:
# https://github.com/VirtualMetabolicHuman 
# 2021-05-12
input_directory = "/Users/pbais/Desktop/Projects/ShuzaoLi/Azimuth/EBI_XML/"

output_directory = "/Users/pbais/Desktop/Projects/ShuzaoLi/Azimuth/azimuth/metabolic_models/"

for filename in os.listdir(input_directory):
    if filename.endswith(".xml"):
        #print(filename)
        f_g = os.path.splitext(filename)[0]
        
        EBI_input = os.path.join(input_directory, filename)
        #print(EBI_input)
        model = cobra.io.read_sbml_model(EBI_input)
        print(f_g)
        
        cpds = []
        anno = {}

        for M in model.metabolites:
            anno[M.id.split("[]")[0]] = M.notes
            cpds.append(metabolite2compound(M))
            

        my_dict = {}
        for M in cpds: my_dict[M.id] = M

        ## compounds to port
        EBI_Compounds = list(my_dict.values())

        # this is the compartmentalized version of reactions
        
        # reactions to port
        EBI_reactions = []
        for R in model.reactions:
            EBI_reactions.append(port_reaction(R))
        
            

        note = """EBI downloaded from https://www.metaboanalyst.ca/faces/docs/Models.xhtml, 2021-06-02"""
        
        ## metabolicModel to export
        MM = metabolicModel()
        MM.id = 'az_EBI_20210602_' + f_g
        MM.meta_data = {
            'species': f_g,
            'version': '',
            'sources': ['https://www.metaboanalyst.ca/faces/docs/Models.xhtml, retrieved 2021-06-02'],
            'status': '',
            'last_update': '20210602',
            'note': note,
        }
       # MM.list_of_pathways = [P.serialize() for P in EBI_pathways]
        MM.list_of_reactions = [R.serialize() for R in  EBI_reactions]
        MM.list_of_compounds = [C.serialize() for C in EBI_Compounds]
        
        #print(filename)
        output_pickle_file = 'metabolicModel_EBI_20210602_' + f_g + '.pickle'
        # pickled object can be imported later, and for Database upload
        EBI_output_pickle = os.path.join(output_directory, 'pickle', output_pickle_file)
        with open(EBI_output_pickle, 'wb') as f:
            pickle.dump(MM.serialize(), f, pickle.HIGHEST_PROTOCOL)

        output_json_file = 'metabolicModel_EBI_20210602_' + f_g  + '.json'
        EBI_output_json = os.path.join(output_directory, 'json', output_json_file)
        s = json.JSONEncoder().encode(MM.serialize())
        with open(EBI_output_json, "w") as O:
            O.write(s)
    else:
        continue